# Jaringan Generatif

Recurrent Neural Networks (RNNs) dan varian sel bergerbang seperti Long Short Term Memory Cells (LSTMs) dan Gated Recurrent Units (GRUs) menyediakan mekanisme untuk pemodelan bahasa, yaitu mereka dapat mempelajari urutan kata dan memberikan prediksi untuk kata berikutnya dalam sebuah urutan. Hal ini memungkinkan kita menggunakan RNN untuk **tugas generatif**, seperti pembuatan teks biasa, terjemahan mesin, dan bahkan pembuatan keterangan gambar.

Dalam arsitektur RNN yang kita bahas di unit sebelumnya, setiap unit RNN menghasilkan keadaan tersembunyi berikutnya sebagai output. Namun, kita juga dapat menambahkan output lain ke setiap unit rekuren, yang memungkinkan kita menghasilkan sebuah **urutan** (yang panjangnya sama dengan urutan aslinya). Selain itu, kita dapat menggunakan unit RNN yang tidak menerima input di setiap langkah, dan hanya mengambil vektor keadaan awal, lalu menghasilkan urutan output.

Di notebook ini, kita akan fokus pada model generatif sederhana yang membantu kita menghasilkan teks. Untuk kesederhanaan, mari kita membangun **jaringan tingkat karakter**, yang menghasilkan teks huruf demi huruf. Selama pelatihan, kita perlu mengambil korpus teks dan membaginya menjadi urutan huruf.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Membangun Kosakata Karakter

Untuk membangun jaringan generatif tingkat karakter, kita perlu membagi teks menjadi karakter individu, bukan kata. Lapisan `TextVectorization` yang telah kita gunakan sebelumnya tidak dapat melakukan itu, jadi kita memiliki dua opsi:

* Memuat teks secara manual dan melakukan tokenisasi 'secara manual', seperti dalam [contoh resmi Keras ini](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* Menggunakan kelas `Tokenizer` untuk tokenisasi tingkat karakter.

Kita akan memilih opsi kedua. `Tokenizer` juga dapat digunakan untuk tokenisasi menjadi kata, sehingga seseorang dapat dengan mudah beralih dari tokenisasi tingkat karakter ke tingkat kata.

Untuk melakukan tokenisasi tingkat karakter, kita perlu memberikan parameter `char_level=True`:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Kami juga ingin menggunakan satu token khusus untuk menandakan **akhir dari urutan**, yang akan kami sebut `<eos>`. Mari kita tambahkan secara manual ke kosakata:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

Sekarang, untuk mengkodekan teks menjadi urutan angka, kita dapat menggunakan:


In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Melatih RNN Generatif untuk Membuat Judul

Cara kita melatih RNN untuk menghasilkan judul berita adalah sebagai berikut. Pada setiap langkah, kita akan mengambil satu judul, yang akan dimasukkan ke dalam RNN, dan untuk setiap karakter input, kita akan meminta jaringan untuk menghasilkan karakter output berikutnya:

![Gambar yang menunjukkan contoh RNN menghasilkan kata 'HELLO'.](../../../../../translated_images/id/rnn-generate.56c54afb52f9781d.webp)

Untuk karakter terakhir dari urutan kita, kita akan meminta jaringan untuk menghasilkan token `<eos>`.

Perbedaan utama antara RNN generatif yang kita gunakan di sini adalah bahwa kita akan mengambil output dari setiap langkah RNN, bukan hanya dari sel terakhir. Hal ini dapat dicapai dengan menentukan parameter `return_sequences` pada sel RNN.

Jadi, selama pelatihan, input ke jaringan akan berupa urutan karakter yang telah dikodekan dengan panjang tertentu, dan outputnya akan berupa urutan dengan panjang yang sama, tetapi digeser satu elemen dan diakhiri dengan `<eos>`. Minibatch akan terdiri dari beberapa urutan seperti itu, dan kita perlu menggunakan **padding** untuk menyelaraskan semua urutan.

Mari kita buat fungsi-fungsi yang akan mengubah dataset untuk kita. Karena kita ingin melakukan padding pada tingkat minibatch, kita akan terlebih dahulu membuat batch dataset dengan memanggil `.batch()`, lalu menggunakan `map` untuk melakukan transformasi. Jadi, fungsi transformasi akan mengambil seluruh minibatch sebagai parameter:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Beberapa hal penting yang kami lakukan di sini:
* Pertama, kami mengekstrak teks sebenarnya dari tensor string
* `text_to_sequences` mengubah daftar string menjadi daftar tensor integer
* `pad_sequences` menambahkan padding pada tensor tersebut hingga panjang maksimum
* Akhirnya, kami melakukan one-hot encoding pada semua karakter, serta melakukan pergeseran dan menambahkan `<eos>`. Kita akan segera melihat mengapa kita membutuhkan karakter yang di-encode dengan one-hot

Namun, fungsi ini bersifat **Pythonic**, yaitu tidak dapat secara otomatis diterjemahkan ke dalam grafik komputasi Tensorflow. Kita akan mendapatkan error jika mencoba menggunakan fungsi ini secara langsung dalam fungsi `Dataset.map`. Kita perlu membungkus panggilan Pythonic ini dengan menggunakan pembungkus `py_function`:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Note**: Membedakan antara fungsi transformasi Pythonic dan Tensorflow mungkin terasa terlalu rumit, dan Anda mungkin bertanya-tanya mengapa kita tidak mengubah dataset menggunakan fungsi Python standar sebelum meneruskannya ke `fit`. Meskipun hal ini tentu saja bisa dilakukan, menggunakan `Dataset.map` memiliki keuntungan besar, karena pipeline transformasi data dieksekusi menggunakan grafik komputasi Tensorflow, yang memanfaatkan perhitungan GPU, dan meminimalkan kebutuhan untuk memindahkan data antara CPU/GPU.

Sekarang kita dapat membangun jaringan generator kita dan mulai pelatihan. Jaringan ini dapat didasarkan pada sel rekuren apa pun yang telah kita bahas di unit sebelumnya (simple, LSTM, atau GRU). Dalam contoh kita, kita akan menggunakan LSTM.

Karena jaringan menerima karakter sebagai input, dan ukuran kosakata cukup kecil, kita tidak memerlukan lapisan embedding, input yang dienkode satu-hot dapat langsung masuk ke sel LSTM. Lapisan output akan menjadi classifier `Dense` yang akan mengubah output LSTM menjadi angka token yang dienkode satu-hot.

Selain itu, karena kita berurusan dengan urutan dengan panjang variabel, kita dapat menggunakan lapisan `Masking` untuk membuat mask yang akan mengabaikan bagian string yang diisi (padded). Ini sebenarnya tidak terlalu diperlukan, karena kita tidak terlalu tertarik pada apa pun yang melampaui token `<eos>`, tetapi kita akan menggunakannya demi mendapatkan pengalaman dengan jenis lapisan ini. `input_shape` akan menjadi `(None, vocab_size)`, di mana `None` menunjukkan urutan dengan panjang variabel, dan bentuk output juga `(None, vocab_size)`, seperti yang dapat Anda lihat dari `summary`:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Menghasilkan Output

Sekarang setelah kita melatih model, kita ingin menggunakannya untuk menghasilkan output. Pertama-tama, kita membutuhkan cara untuk mendekode teks yang direpresentasikan oleh urutan angka token. Untuk melakukan ini, kita bisa menggunakan fungsi `tokenizer.sequences_to_texts`; namun, fungsi ini tidak bekerja dengan baik pada tokenisasi tingkat karakter. Oleh karena itu, kita akan mengambil kamus token dari tokenizer (disebut `word_index`), membangun peta balik, dan menulis fungsi dekode kita sendiri:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Sekarang, mari kita mulai dengan beberapa string `start`, mengkodekannya menjadi sebuah urutan `inp`, dan kemudian pada setiap langkah kita akan memanggil jaringan kita untuk menyimpulkan karakter berikutnya.

Output dari jaringan `out` adalah sebuah vektor dengan `vocab_size` elemen yang merepresentasikan probabilitas dari setiap token, dan kita dapat menemukan nomor token yang paling mungkin dengan menggunakan `argmax`. Kemudian kita menambahkan karakter ini ke daftar token yang dihasilkan, dan melanjutkan proses generasi. Proses menghasilkan satu karakter ini diulangi sebanyak `size` kali untuk menghasilkan jumlah karakter yang diperlukan, dan kita akan menghentikan proses lebih awal ketika `eos_token` ditemukan.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Mengambil sampel output selama pelatihan

Karena kita tidak memiliki metrik yang berguna seperti *akurasi*, satu-satunya cara untuk melihat bahwa model kita semakin baik adalah dengan **mengambil sampel** string yang dihasilkan selama pelatihan. Untuk melakukannya, kita akan menggunakan **callback**, yaitu fungsi yang dapat kita berikan ke fungsi `fit`, dan yang akan dipanggil secara berkala selama pelatihan.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Contoh ini sudah menghasilkan teks yang cukup baik, tetapi masih bisa ditingkatkan dalam beberapa cara:

* **Lebih banyak teks**. Kita hanya menggunakan judul untuk tugas ini, tetapi Anda mungkin ingin bereksperimen dengan teks lengkap. Ingatlah bahwa RNN tidak terlalu baik dalam menangani urutan panjang, jadi masuk akal untuk membaginya menjadi kalimat pendek, atau selalu melatih pada panjang urutan tetap dengan nilai yang telah ditentukan sebelumnya `num_chars` (misalnya, 256). Anda dapat mencoba mengubah contoh di atas menjadi arsitektur seperti itu, menggunakan [tutorial resmi Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/) sebagai inspirasi.

* **LSTM multilayer**. Ada baiknya mencoba 2 atau 3 lapisan sel LSTM. Seperti yang disebutkan dalam unit sebelumnya, setiap lapisan LSTM mengekstrak pola tertentu dari teks, dan dalam kasus generator tingkat karakter, kita dapat mengharapkan tingkat LSTM yang lebih rendah bertanggung jawab untuk mengekstrak suku kata, dan tingkat yang lebih tinggi - untuk kata dan kombinasi kata. Ini dapat diimplementasikan dengan mudah dengan memberikan parameter jumlah lapisan ke konstruktor LSTM.

* Anda juga mungkin ingin bereksperimen dengan **unit GRU** dan melihat mana yang memberikan hasil lebih baik, serta dengan **ukuran lapisan tersembunyi yang berbeda**. Lapisan tersembunyi yang terlalu besar dapat menyebabkan overfitting (misalnya, jaringan akan mempelajari teks secara persis), dan ukuran yang lebih kecil mungkin tidak menghasilkan hasil yang baik.


## Generasi teks lunak dan suhu

Dalam definisi `generate` sebelumnya, kita selalu memilih karakter dengan probabilitas tertinggi sebagai karakter berikutnya dalam teks yang dihasilkan. Hal ini menyebabkan teks sering "berulang" pada urutan karakter yang sama berulang kali, seperti dalam contoh berikut:
```
today of the second the company and a second the company ...
```

Namun, jika kita melihat distribusi probabilitas untuk karakter berikutnya, bisa jadi perbedaan antara beberapa probabilitas tertinggi tidak terlalu besar, misalnya satu karakter memiliki probabilitas 0.2, sementara karakter lain - 0.19, dan sebagainya. Sebagai contoh, ketika mencari karakter berikutnya dalam urutan '*play*', karakter berikutnya bisa saja berupa spasi, atau **e** (seperti dalam kata *player*).

Hal ini membawa kita pada kesimpulan bahwa tidak selalu "adil" untuk memilih karakter dengan probabilitas lebih tinggi, karena memilih karakter dengan probabilitas tertinggi kedua masih bisa menghasilkan teks yang bermakna. Akan lebih bijaksana untuk **mengambil sampel** karakter dari distribusi probabilitas yang diberikan oleh output jaringan.

Pengambilan sampel ini dapat dilakukan menggunakan fungsi `np.multinomial` yang mengimplementasikan apa yang disebut sebagai **distribusi multinomial**. Fungsi yang mengimplementasikan generasi teks **lunak** ini didefinisikan di bawah:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Kami telah memperkenalkan satu parameter lagi yang disebut **temperature**, yang digunakan untuk menunjukkan seberapa ketat kita harus berpegang pada probabilitas tertinggi. Jika temperature adalah 1.0, kita melakukan sampling multinomial yang adil, dan ketika temperature mendekati tak hingga - semua probabilitas menjadi sama, dan kita secara acak memilih karakter berikutnya. Dalam contoh di bawah ini, kita dapat mengamati bahwa teks menjadi tidak bermakna ketika kita meningkatkan temperature terlalu tinggi, dan teks tersebut menyerupai teks "berulang" yang dihasilkan secara kaku ketika mendekati 0.



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan penerjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berupaya untuk memberikan hasil yang akurat, harap diperhatikan bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang berwenang. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa penerjemahan manusia profesional. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang keliru yang timbul dari penggunaan terjemahan ini.
